In [3]:
import os
import subprocess

import torch
from torch.utils.data import Dataset
from torchvision.transforms import ToTensor, Compose
from torchvision import datasets

from nuy_v2_loader import NYUV2Dataset

In [5]:
if not os.path.exists('datasets/diode.tar.gz'):
    subprocess.run(
        ['curl', '-L', '-o', 'datasets/diode.tar.gz', 'http://diode-dataset.s3.amazonaws.com/train.tar.gz'],
        check=True
    )

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0 80.7G    0 31.9M    0     0  44.4M      0  0:30:59 --:--:--  0:30:59 44.4M

KeyboardInterrupt: 

In [ ]:
if not os.path.exists('datasets/nyu_depth_v2'):
    import tensorflow_datasets as tfds
    tfds.load(
        'nyu_depth_v2',
        data_dir='datasets'
    )

In [4]:
NYUV2Dataset("ds", ToTensor(), ToTensor(), 'train', download=True)

Failed to load tar file
Started to creating raw data


FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = '00456.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [ ]:
import h5py
from PIL import Image
import numpy as np

f = h5py.File("ds/nyudepthv2/train/study_0008/00266.h5", "r")

img = f["rgb"]
img = np.stack([(img[channel]) for channel in range(len(img))], axis=2)
# img = np.transpose(img, axes=[1, 2, 0])

depth = f["depth"]

np.max(img), img.shape, depth.shape

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(img)